In [186]:
import pandas as pd
import uuid
from datetime import datetime
import numpy as np
from grakn.client import GraknClient
from tqdm import tqdm
tqdm.pandas()

In [187]:
df=pd.read_excel(r"D:\Threat_intel_services\APT_collections\data\incidents\incidents_final.xlsx")

In [188]:
df['first_seen']= pd.to_datetime(df['first_seen'])
df['last_seen']= pd.to_datetime(df['last_seen'])

In [189]:
def  insert_general_info(df,ent_type,create):
    if create:
        for index,i in df.iterrows():
            df.internal_id_key[index]=str(uuid.uuid4())
            df.stix_id_key[index]=ent_type+'--'+str(uuid.uuid4())
            df.stix_label[index]='NA'
            df.entity_type[index]=ent_type
            df.created=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at_day=(datetime.now()).strftime("%Y-%m-%d")
            df.created_at_month=(datetime.now()).strftime("%Y-%m")
            df.created_at_year=(datetime.now()).strftime("%Y")
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.revoked=False
            df.object_status='NA'
    else:
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
    return df
    

In [190]:
result=insert_general_info(df=df,ent_type='Incident',create=True)
result.fillna('NA',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is t

In [191]:
def insert_one_incident(df,session):
    print(df.first_seen)
    if df.first_seen != 'NA':
       f=df.first_seen.strftime("%Y-%m-%dT%H:%M:%S")
       f_m=df.first_seen.strftime("%Y-%m")
       f_d=df.first_seen.strftime("%Y-%m-%d")
       f_y=df.first_seen.strftime("%Y")
    elif df.first_seen_month != 'NA':
       f_m=df.first_seen_month
    elif df.first_seen_day != 'NA':
       f_d=df.first_seen_day
    else:
       f_d=None
       f_m=None
       f=None
    if df.last_seen != 'NA':
       l=df.last_seen.strftime("%Y-%m-%dT%H:%M:%S")
       l_m=df.last_seen.strftime("%Y-%m")
       l_d=df.last_seen.strftime("%Y-%m-%d")
       l_y=df.last_seen.strftime("%Y")
    elif df.last_seen_month != 'NA':
       l_m=df.last_seen_month
    elif df.last_seen_day != 'NA':
       l_d=df.last_seen_day
    else:
       l_d=None
       l_m=None
       l_y=None
       l=None
#     aliases = list()
#     if df['alias'] != 'NA':            
#         aliases=df.alias.split('\n')

    graql_match_query = f"match $i isa Incident, " \
                        f"has objective \"{df['objective'].strip()}\"; " \
                        f"get;"
    # print(graql_match_query)
    with session.transaction().read() as read_transaction:
        # read_transaction.query(graql_match_query)
        answer_iterator = None
        answer_iterator = read_transaction.query(graql_match_query, infer=True)
        data = answer_iterator.collect_concepts()
                
        if len(data) > 0:
            # Update data.
            # Columns to be updated: object_status, revoked, updated_at, modified, description, alias
            # print("Update intrusion-set...")
            print("Update: " + df['objective'])
            graql_attribute_delete_query = f"match $i isa Incident, " \
                                           f"has objective \"{df['objective'].strip()}\", "\
                                           f"delete $i;"
            update_flg = False
            first_attr_flg = True
            graql_attribute_update_query = f"match $i isa Incident, " \
                                           f"has objective \"{df['objective'].strip()}\"; insert $i"
            
            graql_attribute_update_query += ";"
            print(graql_attribute_update_query)
            if update_flg:
                with session.transaction().write() as write_transaction:
                    # make a write transection with the query
                    write_transaction.query(graql_attribute_delete_query)
                    write_transaction.query(graql_attribute_update_query)
                    # remember to commit at the end
                    write_transaction.commit()           
        else:
            # print("Insert intrusion-set...")
            #print("Insert: " + df['objective'].replace('/"',''))
            df['objective']=df['objective'].replace('/""','').strip()
            graql_insert_query = f"insert $x isa Incident, " \
                                 f"has internal_id_key \"{df['internal_id_key']}\", " \
                                 f"has stix_id_key \"{df['stix_id_key']}\", " \
                                 f"has stix_label \"{df['stix_label']}\", " \
                                 f"has entity_type \"{df['entity_type']}\", " \
                                 f"has created {df['created']}, " \
                                 f"has revoked \"{df['revoked']}\", "\
                                 f"has created_at {df['created_at']}, " \
                                 f"has created_at_day \"{df['created_at_day']}\", " \
                                 f"has created_at_month \"{df['created_at_month']}\", " \
                                 f"has created_at_year \"{df['created_at_year']}\", "\
                                 f"has updated_at {df['updated_at']}, "\
                                 f"has objective \"{df['objective']}\" "\
            
                          
    
            if f is not None:
                graql_insert_query += f", has first_seen {f} "
            if f_d is not None:
                graql_insert_query += f", has first_seen_day \"{f_d}\" "
            if f_m is not None:
                graql_insert_query += f", has first_seen_month \"{f_m}\" "
            if f_y is not None:
                graql_insert_query += f", has first_seen_year \"{f_y}\" "
            if l is not None:
                graql_insert_query += f", has last_seen {l} "
            if l_d is not None:
                graql_insert_query += f", has last_seen_day \"{l_d}\" "
            if l_m is not None:
                graql_insert_query += f", has last_seen_month \"{l_m}\" "
            if l_y is not None:
                graql_insert_query += f", has last_seen_year \"{l_y}\" "
            graql_insert_query += ";"
            print(graql_insert_query)
            with session.transaction().write() as write_transaction:
                # make a write transection with the query
                write_transaction.query(graql_insert_query)
                # remember to commit at the end
                write_transaction.commit()

In [195]:
def load_data_into_grakn(session,input_df):
    print("Malware...")
    # using progress_apply instead of apply so we have a progress bar form tqdm
    # input_df.progress_apply(insert_one_intrusion_set, axis=1, session=session)
    print("Malware...")
    for index,row in input_df.iterrows():
        #insert_one_incident(df=row,session=session)
        try:
            insert_one_incident(df=row,session=session)
        except:
            pass

In [196]:
def build_grakn_graph(input_df, keyspace_name):
    with GraknClient(uri="192.168.83.128:48555") as client:
        with client.session(keyspace = keyspace_name) as session:
            load_data_into_grakn(session,input_df)

In [197]:
build_grakn_graph(input_df=result,keyspace_name='grakn')

Malware...
Malware...
2013-01-01 00:00:00
Update:  APT12 with suspected ties to the People’s Republic of China (PRC) had compromised its networks over the course of the previous four months.
match $i isa Incident, has objective "APT12 with suspected ties to the People’s Republic of China (PRC) had compromised its networks over the course of the previous four months."; insert $i;
1905-07-06 00:00:00
Update: APT12 since at least 2003, when Chinese hackers possibly associated with the PLA infected the networks of a major Taiwanese telecommunications company along with a slew of government websites.
match $i isa Incident, has objective "APT12 since at least 2003, when Chinese hackers possibly associated with the PLA infected the networks of a major Taiwanese telecommunications company along with a slew of government websites."; insert $i;
2009-07-01 00:00:00
Update: The IXESHE attackers have been actively launching highly targeted attacks since at least July 2009.
match $i isa Incident, ha

Russian.
match $i isa Incident, has objective "Spear-phishing attacks to Russian banks
The emails were sent in the name of a large European bank in an 
attempt to social engineer the receiver into trusting the email. The 
emails were quite plain with only a single question in the body and an 
attachment with the name once.rtf. In other cases, we saw a file with 
the name Заявление.rtf attached to an email that was also written in 
Russian."; insert $i;
2018-05-23 00:00:00
NA
NA
2015-04-01 00:00:00
insert $x isa Incident, has internal_id_key "4dd5b293-243c-4333-b441-173065ae6e43", has stix_id_key "Incident--9d08c447-7bf1-4fd2-93ce-7ec626b203f0", has stix_label "NA", has entity_type "Incident", has created 2020-01-10T14:55:40, has revoked "False", has created_at 2020-01-10T14:55:40, has created_at_day "2020-01-10", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-10T14:55:40, has objective "cyber-attack which crippled a French television station in April


match $i isa Incident, has objective "The Sofacy group, also known as APT28, is a well-known threat group that frequently conducts cyber espionage campaigns. Recently, Unit 42 identified a spear phishing e-mail from the Sofacy group that targeted the United States government. The e-mail was sent from a potentially compromised account belonging to the Ministry of Foreign Affairs of another government entity and carried the Carberp variant of the Sofacy Trojan. The developer implemented a clever persistence mechanism in the Trojan, one which had not been observed in previous attacks. The focus of this blog will be on the attacks and the infrastructure associated with Sofacy using the new persistence mechanism as a correlation point."; insert $i;
2016-08-01 00:00:00
Update: Phishing emails sent to users of its database claiming to be 
official WADA communications requesting their login details.
match $i isa Incident, has objective "Phishing emails sent to users of its database claiming t

In [198]:
f=True
a='hello;howare u'
a+=',ello'
if f:
    a=a.replace(',','')
    f=False
a+='fine'

In [199]:
a

'hello;howare uellofine'